In [270]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image


In [271]:
def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches





def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def patch_to_img(imdim, w, h, patches):
    if(len(imdim) < 3):
        im=np.zeros([imdim[0],imdim[1]])
    else:
            im=np.zeros([imdim[0],imdim[1],imdim[2]])
    imgwidth = imdim[0]
    imgheight = imdim[0]
    
    idx = 0

    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h,:] =patches[idx]
            idx = idx + 1
            
    return im

In [272]:
def get_neighbours_index(i,j,n_r,n_c):
    i=int(i);
    j=int(j);
    n_r=int(n_r)
    n_c=int(n_c)
    if (i==0):
            if(j==0):
                return([j*n_r+i+1,(j+1)*n_r+i,(j+1)*n_r+i+1])
            else:
                if(j==n_c-1):
                    return([(j-1)*n_r+i,(j-1)*n_r+i+1,j*n_r+i+1])
                else:
                    return([(j-1)*n_r+i,(j+1)*n_r+i,(j-1)*n_r+i+1,j*n_r+i+1,(j+1)*n_r+i+1])
    else: 
            
        if (i==n_r-1):
                if(j==0):
                    return([j*n_r+i-1,(j+1)*n_r+i-1,(j+1)*n_r+i])
                else:
                    if(j==n_c-1):
                        return([(j-1)*n_r+i-1,j*n_r+i-1,(j-1)*n_r+i])
                    else:
                        return([(j-1)*n_r+i-1,j*n_r+i-1,(j+1)*n_r+i-1,(j-1)*n_r+i,(j+1)*n_r+i])
                
        else: 
            if(j==0):
                return([j*n_r+i-1,(j+1)*n_r+i-1,(j+1)*n_r+i,j*n_r+i+1,(j+1)*n_r+i+1])
            else:
                if(j==n_c-1):
                    return([j*n_r+i-1,(j-1)*n_r+i-1,(j-1)*n_r+i,j*n_r+i+1,(j-1)*n_r+i+1])
                else:
                    return([j*n_r+i-1,(j-1)*n_r+i-1,(j-1)*n_r+i,j*n_r+i+1,(j-1)*n_r+i+1,(j+1)*n_r+i-1,(j+1)*n_r+i,(j+1)*n_r+i+1])
            
                
                

    
    
    

In [273]:
val=get_neighbours_index(5,2,6,6)
print(val)

[10, 16, 22, 11, 23]


In [274]:
def fill_the_gaps_on_patches(im_patches,im_dim,w,h):
    n_patch_row=int(im_dim[0]/w);
    n_patch_col=int(im_dim[1]/h);
    print('number of rows')
    print(n_patch_row)
    
    means_list=[]
    for k in range(len(im_patches)):
        mean=np.mean(im_patches[k]);
        print('mean')
        print(mean)
        means_list.append(mean)
    
    
    for k in range(len(im_patches)):
        print('indice di patch')
        print(k)
        index_row=int(k%n_patch_col);
       
      
        index_col=int(k//n_patch_col);
       
        
        patch=im_patches[k];
        mean=np.mean(patch);
        list_of_index=get_neighbours_index(index_row,index_col,n_patch_row,n_patch_col)
        means_neighbours=[]
      
        for i1 in range(len(list_of_index)):
            list_of_index[i1]
            means_neighbours.append(means_list[list_of_index[i1]]);

            
                                        
        count_white=0;
        count_black=0;
        for i in range(len(means_neighbours)):
            if(means_neighbours[i]==255):
                count_white=count_white+1;
                
            if(means_neighbours[i]==0):
                count_black=count_black+1;
        
        if(means_list[k]==0):
            if (len(means_neighbours)==8):
                    if (count_white>5):
                        im_patches[k]=np.ones([16,16,3])*255
                        print('fatto')
            if (len(means_neighbours)==3):     
                    if (count_white>2):
                        im_patches[k]=np.ones([16,16,3])*255
                        print('fatto')
            if (len(means_neighbours)==5):      
                    if (count_white>4):
                        im_patches[k]=np.ones([16,16,3])*255
                        print('fatto')
        if(means_list[k]==255):
            if (len(means_neighbours)==8):
                    if (count_black>7):
                        im_patches[k]=np.zeros([16,16,3])*255
                        print('fatto')
            if (len(means_neighbours)==3):     
                    if (count_black>2):
                        im_patches[k]=np.zeros([16,16,3])*255
                        print('fatto')
            if (len(means_neighbours)==5):      
                    if (count_black>4):
                        im_patches[k]=np.zeros([16,16,3])*255
                        print('fatto')
    
    return im_patches
                
        
                   #first row
                
         



In [275]:
def fill_the_gaps(img,w,h,im_dim):
    im_patches=img_crop(img,w,h);
    new_im_patches=fill_the_gaps_on_patches(im_patches,im_dim,w,h);
    new_im=patch_to_img(im_dim, w, h, new_im_patches)
    return new_im;
    

In [276]:
root_dir = "training/"
w=16;
h=16;
image_dir = root_dir + "images/"
files = os.listdir(image_dir)
files = list(np.sort(files))
n = min(20, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs_output = [load_image(image_dir + files[i]) for i in range(n)]

Loading 1 images


In [277]:
new_imgs_output=imgs_output;
print(imgs_output[0].shape)
im_dim=imgs_output[0].shape;

print(im_dim)
for i in range(len(imgs_output)):
    new_imgs_output[i]=fill_the_gaps(imgs_output[i],w,h,im_dim);
    print(new_imgs_output[i].shape)
    img=new_imgs_output[i].astype(np.uint8)
    
   


    Image.fromarray(img).save('prediction_corrected' + '%.3d' % i + '.png')

(608, 608, 3)
(608, 608, 3)
number of rows
38
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
255.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
255.0
mean
255.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mea

mean
255.0
mean
0.0
mean
0.0
mean
255.0
mean
255.0
mean
0.0
mean
255.0
mean
255.0
mean
255.0
mean
0.0
mean
0.0
mean
255.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
255.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
255.0
mean
0.0
mean
255.0
mean
0.0
mean
0.0
mean
0.0
mean
255.0
mean
255.0
mean
0.0
mean
0.0
mean
255.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
255.0
mean
0.0
mean
0.0
mean
0.0
mean
255.0
mean
255.0
mean
255.0
mean
0.0
mean
255.0
mean
255.0
mean
0.0
mean
0.0
mean
255.0
mean
255.0
mean
255.0
mean
255.0
mean
255.0
mean
255.0
mean
255.0
mean
0.0
mean
0.0
mean
255.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
0.0
mean
255.0
mean
255.0
mean
255.0
mean
0.0
mean
255.0
mean
255.0
mean
255.0
mean
25

indice di patch
934
indice di patch
935
indice di patch
936
indice di patch
937
indice di patch
938
indice di patch
939
indice di patch
940
indice di patch
941
indice di patch
942
indice di patch
943
indice di patch
944
indice di patch
945
indice di patch
946
indice di patch
947
indice di patch
948
indice di patch
949
indice di patch
950
indice di patch
951
indice di patch
952
indice di patch
953
indice di patch
954
indice di patch
955
indice di patch
956
indice di patch
957
indice di patch
958
indice di patch
959
indice di patch
960
indice di patch
961
indice di patch
962
indice di patch
963
indice di patch
964
indice di patch
965
indice di patch
966
indice di patch
967
indice di patch
968
indice di patch
969
fatto
indice di patch
970
fatto
indice di patch
971
indice di patch
972
indice di patch
973
indice di patch
974
indice di patch
975
indice di patch
976
indice di patch
977
indice di patch
978
indice di patch
979
indice di patch
980
indice di patch
981
indice di patch
982
indice d